In [1]:
import pandas as pd

In [2]:
df = pd.read_parquet("s3://group1projectbucket2/intermediate/final_sampled_flight_data_all_years (1).parquet")

/home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/fsspec/registry.py:283: UserWarning: Your installed version of s3fs is very old and known to cause
severe performance issues, see also https://github.com/dask/dask/issues/10276

To fix, you should specify a lower version bound on s3fs, or
update the current installation.

  warnings.warn(s3_msg)


In [3]:
print(df.shape)

(740204, 64)


In [4]:
df.columns

Index(['FlightDate', 'Airline', 'Origin', 'Dest', 'Cancelled', 'Diverted',
       'CRSDepTime', 'DepTime', 'DepDelayMinutes', 'DepDelay', 'ArrTime',
       'ArrDelayMinutes', 'AirTime', 'CRSElapsedTime', 'ActualElapsedTime',
       'Distance', 'Year', 'Quarter', 'Month', 'DayofMonth', 'DayOfWeek',
       'Marketing_Airline_Network', 'Operated_or_Branded_Code_Share_Partners',
       'DOT_ID_Marketing_Airline', 'IATA_Code_Marketing_Airline',
       'Flight_Number_Marketing_Airline', 'Operating_Airline',
       'DOT_ID_Operating_Airline', 'IATA_Code_Operating_Airline',
       'Tail_Number', 'Flight_Number_Operating_Airline', 'OriginAirportID',
       'OriginAirportSeqID', 'OriginCityMarketID', 'OriginCityName',
       'OriginState', 'OriginStateFips', 'OriginStateName', 'OriginWac',
       'DestAirportID', 'DestAirportSeqID', 'DestCityMarketID', 'DestCityName',
       'DestState', 'DestStateFips', 'DestStateName', 'DestWac', 'DepDel15',
       'DepartureDelayGroups', 'DepTimeBlk', 'TaxiOu

In [5]:
rename_dict = {
    "CRSArrTime": "SchdArrTime",
    "CRSDepTime": "SchdDepTime",
    "CRSElapsedTime": "SchdElapsedTime"
}

# Rename columns
df.rename(columns=rename_dict, inplace=True)

# Show updated DataFrame
print(df)

       FlightDate                 Airline Origin Dest  Cancelled  Diverted  \
0      2018-01-01    Alaska Airlines Inc.    LAX  SEA      False     False   
1      2018-01-01  Southwest Airlines Co.    DEN  SFO      False     False   
2      2018-01-01   United Air Lines Inc.    DEN  SFO      False     False   
3      2018-01-01    Delta Air Lines Inc.    OAK  SLC      False     False   
4      2018-01-01   United Air Lines Inc.    DFW  LAX      False     False   
...           ...                     ...    ...  ...        ...       ...   
748268 2022-07-31  Frontier Airlines Inc.    LAS  PHL      False     False   
748270 2022-07-31    Delta Air Lines Inc.    PDX  ATL      False     False   
748271 2022-07-31        Spirit Air Lines    LAX  DFW      False     False   
748272 2022-07-31    Delta Air Lines Inc.    LAS  ATL      False     False   
748273 2022-07-31  Southwest Airlines Co.    SMF  LAS      False     False   

        SchdDepTime DepTime  DepDelayMinutes  DepDelay  ...  Sc

In [6]:
df.columns

Index(['FlightDate', 'Airline', 'Origin', 'Dest', 'Cancelled', 'Diverted',
       'SchdDepTime', 'DepTime', 'DepDelayMinutes', 'DepDelay', 'ArrTime',
       'ArrDelayMinutes', 'AirTime', 'SchdElapsedTime', 'ActualElapsedTime',
       'Distance', 'Year', 'Quarter', 'Month', 'DayofMonth', 'DayOfWeek',
       'Marketing_Airline_Network', 'Operated_or_Branded_Code_Share_Partners',
       'DOT_ID_Marketing_Airline', 'IATA_Code_Marketing_Airline',
       'Flight_Number_Marketing_Airline', 'Operating_Airline',
       'DOT_ID_Operating_Airline', 'IATA_Code_Operating_Airline',
       'Tail_Number', 'Flight_Number_Operating_Airline', 'OriginAirportID',
       'OriginAirportSeqID', 'OriginCityMarketID', 'OriginCityName',
       'OriginState', 'OriginStateFips', 'OriginStateName', 'OriginWac',
       'DestAirportID', 'DestAirportSeqID', 'DestCityMarketID', 'DestCityName',
       'DestState', 'DestStateFips', 'DestStateName', 'DestWac', 'DepDel15',
       'DepartureDelayGroups', 'DepTimeBlk', 'Taxi

In [7]:
import pandas as pd
import numpy as np

# Assuming df is your Pandas DataFrame

# Ensure SchdDepTime is a string with 4 digits (left pad with zeros if needed)
df['SchdDepTime'] = df['SchdDepTime'].astype(str).str.zfill(4)

# Extract SchdDepHour (first two digits of SchdDepTime)
df['SchdDepHour'] = df['SchdDepTime'].str[:2].astype(int)

# Categorize into different time periods of the day
df['SchdDepTimeOfDay'] = np.select(
    [
        (df['SchdDepHour'] >= 0) & (df['SchdDepHour'] < 6),
        (df['SchdDepHour'] >= 6) & (df['SchdDepHour'] < 12),
        (df['SchdDepHour'] >= 12) & (df['SchdDepHour'] < 18)
    ],
    ['Night', 'Morning', 'Afternoon'],
    default='Evening'
)

# Categorize DelayCategory based on ArrDelayMinutes
df['DelayCategory'] = np.select(
    [
        df['ArrDelayMinutes'] < 15,
        (df['ArrDelayMinutes'] >= 15) & (df['ArrDelayMinutes'] < 60),
        (df['ArrDelayMinutes'] >= 60) & (df['ArrDelayMinutes'] < 120),
        (df['ArrDelayMinutes'] >= 120) & (df['ArrDelayMinutes'] < 180)
    ],
    ['No Delay', 'Short Delay', 'Medium Delay', 'Long Delay'],
    default='Very Long Delay'
)


In [8]:
df.columns

Index(['FlightDate', 'Airline', 'Origin', 'Dest', 'Cancelled', 'Diverted',
       'SchdDepTime', 'DepTime', 'DepDelayMinutes', 'DepDelay', 'ArrTime',
       'ArrDelayMinutes', 'AirTime', 'SchdElapsedTime', 'ActualElapsedTime',
       'Distance', 'Year', 'Quarter', 'Month', 'DayofMonth', 'DayOfWeek',
       'Marketing_Airline_Network', 'Operated_or_Branded_Code_Share_Partners',
       'DOT_ID_Marketing_Airline', 'IATA_Code_Marketing_Airline',
       'Flight_Number_Marketing_Airline', 'Operating_Airline',
       'DOT_ID_Operating_Airline', 'IATA_Code_Operating_Airline',
       'Tail_Number', 'Flight_Number_Operating_Airline', 'OriginAirportID',
       'OriginAirportSeqID', 'OriginCityMarketID', 'OriginCityName',
       'OriginState', 'OriginStateFips', 'OriginStateName', 'OriginWac',
       'DestAirportID', 'DestAirportSeqID', 'DestCityMarketID', 'DestCityName',
       'DestState', 'DestStateFips', 'DestStateName', 'DestWac', 'DepDel15',
       'DepartureDelayGroups', 'DepTimeBlk', 'Taxi

In [9]:
df["IsWeekend"] = df["DayOfWeek"].apply(lambda x: 1 if x in [6, 7] else 0)

In [10]:
columns_to_remove = [
    # Columns directly related to the target (leakage)
    "ArrDelayMinutes", "ArrDelay", "ArrDel15", "ArrivalDelayGroups", "ArrTimeBlk",
    
    # Columns with unique identifiers or irrelevant information
    "Tail_Number", "Flight_Number_Marketing_Airline", "Flight_Number_Operating_Airline",
    "OriginAirportSeqID", "DestAirportSeqID", "OriginCityMarketID", "DestCityMarketID",
    "OriginStateFips", "DestStateFips", "OriginWac", "DestWac", "DOT_ID_Marketing_Airline",
    "DOT_ID_Operating_Airline","SchdDepTimeOfDay",
    
    # Redundant columns (highly correlated with others)
    "DepDelay", "DepDel15", "DepartureDelayGroups", "DepTimeBlk","DivAirportLandings", "DistanceGroup",
    
    # Columns with duplicate or similar information
    "OriginAirportID", "DestAirportID",  # Use Origin and Dest instead
    "IATA_Code_Marketing_Airline", "IATA_Code_Operating_Airline",  # Use Airline and Operating_Airline instead
    "OriginState", "DestState",  # Use OriginStateName and DestStateName instead
    "Marketing_Airline_Network", "Operated_or_Branded_Code_Share_Partners",  # Use Airline and Operating_Airline instead
    "DepTime", "DepDelayMinutes", "ArrTime", "AirTime",  # Numerical columns
    "Operating_Airline", "OriginCityName", "DestCityName",  # Categorical columns
    "FlightDate","SchdDepTime","SchdElapsedTime","ActualElapsedTime","Distance"
]

In [11]:
df = df.drop(columns=columns_to_remove)

In [12]:
df.columns

Index(['Airline', 'Origin', 'Dest', 'Cancelled', 'Diverted', 'Year', 'Quarter',
       'Month', 'DayofMonth', 'DayOfWeek', 'OriginStateName', 'DestStateName',
       'TaxiOut', 'WheelsOff', 'WheelsOn', 'TaxiIn', 'SchdArrTime', 'Day',
       'Hour', 'Date', 'SchdDepHour', 'DelayCategory', 'IsWeekend'],
      dtype='object')

In [13]:
col_to_remove=["Cancelled","Diverted", "Day", "Hour", "Date"]
df = df.drop(columns=col_to_remove)

In [14]:
df.columns

Index(['Airline', 'Origin', 'Dest', 'Year', 'Quarter', 'Month', 'DayofMonth',
       'DayOfWeek', 'OriginStateName', 'DestStateName', 'TaxiOut', 'WheelsOff',
       'WheelsOn', 'TaxiIn', 'SchdArrTime', 'SchdDepHour', 'DelayCategory',
       'IsWeekend'],
      dtype='object')

In [15]:
df=df.drop(columns=["OriginStateName", "DestStateName"])

In [16]:
df.columns

Index(['Airline', 'Origin', 'Dest', 'Year', 'Quarter', 'Month', 'DayofMonth',
       'DayOfWeek', 'TaxiOut', 'WheelsOff', 'WheelsOn', 'TaxiIn',
       'SchdArrTime', 'SchdDepHour', 'DelayCategory', 'IsWeekend'],
      dtype='object')

In [17]:
bins = [0, 6, 12, 18, 24]
labels = ['Night', 'Morning', 'Afternoon', 'Evening']
df['SchdDepTimeOfDay'] = pd.cut(df['SchdDepHour'], bins=bins, labels=labels, right=False)

In [18]:
print(df['Airline'])  # Prints the entire column


0           Alaska Airlines Inc.
1         Southwest Airlines Co.
2          United Air Lines Inc.
3           Delta Air Lines Inc.
4          United Air Lines Inc.
                   ...          
748268    Frontier Airlines Inc.
748270      Delta Air Lines Inc.
748271          Spirit Air Lines
748272      Delta Air Lines Inc.
748273    Southwest Airlines Co.
Name: Airline, Length: 740204, dtype: object


In [19]:
categorical_cols = ['Airline', 'Origin', 'Dest', 'SchdDepTimeOfDay']
numerical_cols = [
    "SchdDepHour", "Year", "Quarter", "Month", "DayofMonth", "DayOfWeek",
    "TaxiOut", "WheelsOff", "WheelsOn", "TaxiIn", "SchdArrTime", "IsWeekend"
]

In [20]:
X = df.drop(columns=["DelayCategory"])

In [21]:
X.columns

Index(['Airline', 'Origin', 'Dest', 'Year', 'Quarter', 'Month', 'DayofMonth',
       'DayOfWeek', 'TaxiOut', 'WheelsOff', 'WheelsOn', 'TaxiIn',
       'SchdArrTime', 'SchdDepHour', 'IsWeekend', 'SchdDepTimeOfDay'],
      dtype='object')

In [22]:
y = df["DelayCategory"]  # Target

In [24]:
len(X.columns)

16

In [25]:
import joblib


In [26]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, OneHotEncoder, StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestClassifier
from sklearn.impute import SimpleImputer
from sklearn.metrics import classification_report

In [27]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [28]:
# Preprocessing pipeline
preprocessor = ColumnTransformer(
    transformers=[
        ("num", StandardScaler(), numerical_cols),  # Scale numerical features
        ("cat", OneHotEncoder(handle_unknown="ignore"), categorical_cols),  # One-hot encode categorical features
    ]
)

In [29]:
# Fit on training data
X_train = preprocessor.fit_transform(X_train)

In [30]:
import joblib

# Save the preprocessor as a .joblib file
joblib.dump(preprocessor, "preprocessor.joblib")

print("Preprocessor saved as preprocessor.joblib")


Preprocessor saved as preprocessor.joblib


In [31]:
# Transform test data
X_test = preprocessor.transform(X_test)

In [32]:
# Encode target labels
label_encoder = LabelEncoder()
y_train = label_encoder.fit_transform(y_train)

In [33]:
import joblib

# Save the label encoder as a .joblib file
joblib.dump(label_encoder, "label_encoder.joblib")

print("Label encoder saved as label_encoder.joblib")


Label encoder saved as label_encoder.joblib


In [34]:
# Transform test labels
y_test = label_encoder.transform(y_test)

In [35]:
print(X_train.shape,y_train.shape,X_test.shape,y_test.shape)

(592163, 818) (592163,) (148041, 818) (148041,)


In [36]:
label_mapping = {class_label: index for index, class_label in enumerate(label_encoder.classes_)}
label_mapping

{'Long Delay': 0,
 'Medium Delay': 1,
 'No Delay': 2,
 'Short Delay': 3,
 'Very Long Delay': 4}

In [37]:
for index, class_label in enumerate(label_encoder.classes_):
    print(f"Original Label: {class_label} -> Encoded Value: {index}")


Original Label: Long Delay -> Encoded Value: 0
Original Label: Medium Delay -> Encoded Value: 1
Original Label: No Delay -> Encoded Value: 2
Original Label: Short Delay -> Encoded Value: 3
Original Label: Very Long Delay -> Encoded Value: 4


In [38]:
%pip install xgboost pandas scikit-learn

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.5/4.5 MB 109.6 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [39]:
from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score, classification_report

/home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/xgboost/core.py:265: FutureWarning: Your system has an old version of glibc (< 2.28). We will stop supporting Linux distros with glibc older than 2.28 after **May 31, 2025**. Please upgrade to a recent Linux distro (with glibc 2.28+) to use future versions of XGBoost.
Note: You have installed the 'manylinux2014' variant of XGBoost. Certain features such as GPU algorithms or federated learning are not available. To use these features, please upgrade to a recent Linux distro with glibc 2.28+, and install the 'manylinux_2_28' variant.
  warnings.warn(


In [40]:
from xgboost import XGBClassifier

In [ ]:
# Initialize the model
xgb = XGBClassifier(random_state=42, use_label_encoder=False, eval_metric='logloss')

# Train the model
xgb.fit(X_train, y_train)

# Predict on the test set
y_pred_xgb = xgb.predict(X_test)

# Evaluate accuracy
accuracy_xgb = accuracy_score(y_test, y_pred_xgb)
print(f"XGBoost Accuracy: {accuracy_xgb:.4f}")
print(classification_report(y_test, y_pred_xgb))

/home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/xgboost/core.py:158: UserWarning: [07:15:40] WARNING: /workspace/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)


In [ ]:
from sklearn.metrics import confusion_matrix, classification_report
import seaborn as sns
import matplotlib.pyplot as plt

# Compute confusion matrix
cm = confusion_matrix(y_test, y_pred_xgb)

# Retrieve class labels from the label encoder
class_labels = label_encoder.classes_  # Gets the original category names

# Print confusion matrix
print("Confusion Matrix:")
print(cm)

# Plot confusion matrix
plt.figure(figsize=(6, 5))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', xticklabels=class_labels, yticklabels=class_labels)
plt.xlabel('Predicted')
plt.ylabel('Actual')
plt.title('Confusion Matrix')
plt.show()


In [ ]:
import joblib

# Save the trained model as a joblib file
model_filename = "xgboost_model.joblib"
joblib.dump(xgb, model_filename)

print(f"Model saved as {model_filename}")


In [ ]:
import pickle

# Save the trained model as a pickle file
model_filename = "xgboost_model.pkl"
with open(model_filename, "wb") as file:
    pickle.dump(xgb, file)

print(f"Model saved as {model_filename}")


In [ ]:
from sklearn.metrics import f1_score

# Calculate the F1-score
f1 = f1_score(y_test, y_pred_xgb, average="weighted")  # Use 'weighted' for multiclass classification

print(f"F1 Score: {f1:.4f}")
